In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df = pd.read_csv('CR_COST_FC.csv', converters={'PARTY_ID': str, 'COM_ID': str, 'CNTR_SIZE': str})
df['POD'] = pd.to_datetime(df['POD'])
df['ENCODED_TYPE'] = df['ENCODED_TYPE'].fillna(-1).astype(int)
df = df.dropna(subset=['ENCODED_TYPE'])
df['RATE'] = df['RATE'].fillna(-1).astype(int)
df = df.dropna(subset=['RATE'])
df['ENCODED_TYPE'] = df['ENCODED_TYPE'].astype(int)


df.head()
df.info()
print(f'Dataset size: {df.shape}')

<h4>Checking if dataset still contains any NAN values after datacleaning using SQL</h4>

In [ ]:
if df.isna().any().any():
    print('Dataframe contains NaN values')
else:
    print('Dataframe does not contain NaN values')

<h4>Removing rows contains NAN values</h4>

In [ ]:
df_clean = df.dropna().reset_index(drop=True)
df_clean.info()

<h3>Casting All to INT32 or INT64 Type</h3>

In [ ]:
df_clean['CNTR_SIZE'] = df_clean['CNTR_SIZE'].astype(np.int32)
df_clean['RATE'] = df_clean['RATE'].astype(np.int32)
df_clean['PARTY_ID_EN'] = df_clean['PARTY_ID_EN'].astype(np.int32)
df_clean['POD_ID_EN'] = df_clean['POD_ID_EN'].astype(np.int64)
df_clean['ETA_ETD_NO'] = df_clean['ETA_ETD_NO'].astype(np.int32)
df_clean.info()
df_clean.head()

<h4>Rearrange columns where ID first then label</h4>

In [ ]:
sel_col = ['COM_ID','CSL_ID', 'CNTR_ID','ENCODED_TERM','COST_TERM','POD_ID','ETD_POL_D','PARTY_ID',
           'PARTY_ID_EN', 'PARTY_NAME','POD_ID_EN','ETA_ETD_NO','POD',
           'CNTR_SIZE','ENCODED_TYPE','CNTR_TYPE','RATE']

df_fc = df_clean[sel_col]
df_fc.head()
df_fc.info()

In [ ]:
# interpolate missing values
df_fc['POD'].dt.year.unique()

In [ ]:
# Sorting by date so can interpolate the missing dates while using cost median 
df_fc = df_fc.sort_values(by='POD').reset_index(drop=True)
df_fc.head()
df_fc['POD'].dt.year.unique()

<p>Removing year 2002 as data before 2005 was used for data testing. Hence, it is not relevant.</p>

In [ ]:
# filter out rows where the year is 2002
df_filtered = df_fc[df_fc['POD'].dt.year != 2002]
df_filtered.head()
df_filtered.info()

# Checking if year 2002 is removed
df_filtered['POD'].dt.year.unique()

<h3>Filtering by most export count, container size, container type and focus in on most used shipping vendor quotations.</h3>

In [ ]:
# View which is the most popular port using numpy
port_id, port_count = np.unique(df_filtered["POD_ID"], return_counts=True)
count_sort_ind = np.argsort(-port_count)
print(list(zip(port_id[count_sort_ind], port_count[count_sort_ind])))
print(len(port_id))

<p>From the above, AUCKLAND are the biggest export port from Singapore based on FC Cost Term</p>

In [ ]:
# Working with Auckland
df_largest_port = df_filtered[df_filtered['POD_ID'] == 'AUCKLAND'].reset_index(drop=True)
df_largest_port.head()
df_largest_port.info()

In [ ]:
# View which is the most popular container size using numpy
port_size, port_size_count = np.unique(df_largest_port["CNTR_SIZE"], return_counts=True)
count_sort_ind = np.argsort(-port_size_count)
print(list(zip(port_size[count_sort_ind], port_size_count[count_sort_ind])))
print(len(port_size))

<p>From the filter above, container exported are mostly size 40 and exported container sizes to Auckland is only size 40 and size 20.</p>

In [ ]:
# Working with container size 40
df_40 = df_largest_port[df_largest_port['CNTR_SIZE'] == 40].reset_index(drop=True)
df_40.head()
df_40.info()

In [ ]:
# View which is the most popular container type using numpy
port_id, port_type_count = np.unique(df_40["CNTR_TYPE"], return_counts=True)
count_sort_ind = np.argsort(-port_type_count)
print(list(zip(port_id[count_sort_ind], port_type_count[count_sort_ind])))
print(len(port_id))

<p>For the filter above, we can see that for Auckland, container size 40, consist of container type HC, HC NOR while a few by GP and HC. HC is has the highest count for container type. </p>

In [ ]:
# Working with container type HC
df_hc = df_40[df_40['CNTR_TYPE'] == 'HC'].reset_index(drop=True)
df_hc.head()
df_hc.info()

In [ ]:
# View which is the most used shipping vendor using numpy
vendor_id, vendor_count = np.unique(df_hc["PARTY_NAME"], return_counts=True)
count_sort_ind = np.argsort(-vendor_count)
print(list(zip(vendor_id[count_sort_ind], vendor_count[count_sort_ind])))
print(len(vendor_id))

<p>From the filter above we can observe that exports for Auckland, container size 40 and type HC, the company mostly uses 'MONDIALE FREIGHT SERVICES LIMITED-AUCKLAND' quotation for all their exports.</p>
<p>The filted dataframe only consist of 1 vendor. Hence, we do not need to filter anymore and proceed with performing modeling.</p> 
<p>After focusing in a party id, we do need ot interpolate the dates as the dates are not in series which could be a problem when performing time series models like ARIMA or Prophet.</p>

In [ ]:
# Checking for negative cost rate before interpolate
for values in df_hc['RATE']:
    if values <= 0:
        print(values)

In [80]:
# fill missing dates and values using forward fill
df_hc_filled = df_hc.groupby('POD').apply(lambda x: x.set_index('ETD_POL_D').asfreq('D').fillna(method='ffill')).reset_index()

# interpolate the missing values
df_hc_interpolated = df_hc_filled.groupby('POD').apply(lambda x: x.interpolate(method='linear')).reset_index(drop=True)

# plot the interpolated data
df_hc_interpolated.plot(x='ETD_POL_D', y='RATE')
plt.show()

# check the info
df_hc_interpolated.info()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas\_libs\tslibs\conversion.pyx", line 530, in pandas._libs.tslibs.conversion._convert_str_to_tsobject
  File "pandas\_libs\tslibs\parsing.pyx", line 318, in pandas._libs.tslibs.parsing.parse_datetime_string
  File "c:\Users\Aloysius Wong\Documents\GitHub\Cost-Rate-Analysis\env\lib\site-packages\dateutil\parser\_parser.py", line 1368, in parse
    return DEFAULTPARSER.parse(timestr, **kwargs)
  File "c:\Users\Aloysius Wong\Documents\GitHub\Cost-Rate-Analysis\env\lib\site-packages\dateutil\parser\_parser.py", line 643, in parse
    raise ParserError("Unknown string format: %s", timestr)
dateutil.parser._parser.ParserError: Unknown string format: NZAKL

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Aloysius Wong\Documents\GitHub\Cost-Rate-Analysis\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, sel

In [ ]:
# from scipy.stats import boxcox

# # Doing symmetric distribution to determine to agg duplicated cost rate on the same date using mean or median
# lambda_ = boxcox(interpolated_data['RATE'])
# print(lambda_)
# plt.hist(lambda_, bins=10)
# plt.show()

<p>Negatively skewed data, it is usually recommended to use the median as the measure of central tendency instead of the mean. </p>

In [ ]:
# # perform the aggregation and reset the index
# df_agg = interpolated_data.groupby('POD').agg(aggregation).reset_index()
# df_agg.head()
# df_agg.info()